In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from FrCHFM import compute_frchfm_gpu, convert_to_polar, radial_basis_function_gpu
import os
from tqdm import tqdm
import cupy as cp
from concurrent.futures import ThreadPoolExecutor, as_completed

[NbConvertApp] Converting notebook FrCHFM.ipynb to python
[NbConvertApp] Writing 11080 bytes to FrCHFM.py


In [2]:
def rotate_image_fixed_size(image, angle):

    """
    Rotates the image by a given angle around its center, maintaining the original size with potential cropping.
    Parameters:

        image (numpy array): Input image to rotate.

        angle (float): Angle by which to rotate the image.
    Returns:
        rotated_image (numpy array): Rotated image, constrained to original dimensions.
    """
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)

    # Compute the rotation matrix and apply rotation
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, M, (w, h))

    return rotated_image


In [3]:
def compute_frchfm_difference(frchfm_original, frchfm_rotated):
    """
    Calculates the mean absolute difference between the magnitudes of the original and rotated FrCHFM matrices.
    
    Parameters:
        frchfm_original (2D numpy array): FrCHFM matrix of the original image.
        frchfm_rotated (2D numpy array): FrCHFM matrix of the rotated image.
        
    Returns:
        mean_difference (float): Mean absolute difference between the magnitudes.
    """
    return np.mean(np.abs(np.abs(frchfm_original) - np.abs(frchfm_rotated)))

In [4]:
def rotate_and_compare_gpu(image, max_degree, fractional_parameter_t, angles=[45, 90, 135, 180, 225, 270, 315, 360]):
    """
    Rotates an image by specified angles, computes FrCHFM for each, and measures rotation invariance using GPU.
    
    Parameters:
        image (numpy array): The original image to test.
        max_degree (int): Maximum degree for the FrCHFM calculation.
        fractional_parameter_t (float): Fractional parameter for FrCHFM.
        angles (list): List of angles to rotate the image for testing.
        
    Returns:
        results (dict): A dictionary where keys are angles and values are tuples (is_invariant, deviation).
                        - `is_invariant` (bool): Whether the FrCHFM is invariant at this angle.
                        - `deviation` (float): Mean absolute difference in FrCHFM magnitudes.
    """
    # Transfer image to the GPU and compute FrCHFM for the original
    frchfm_original = compute_frchfm_gpu(image, max_degree, fractional_parameter_t)
    results = {}

    for angle in angles:
        # Rotate image on CPU and then transfer to GPU
        rotated_image = rotate_image_fixed_size(image, angle)
        frchfm_rotated = compute_frchfm_gpu(rotated_image, max_degree, fractional_parameter_t)

        # Calculate the mean absolute difference as a measure of deviation (using GPU arrays)
        deviation = cp.mean(cp.abs(cp.abs(cp.array(frchfm_original)) - cp.abs(cp.array(frchfm_rotated))))
        
        # Determine invariance by checking if deviation is within tolerance
        is_invariant = deviation.get() < 1e-6  # .get() to bring the result back to CPU
        results[angle] = (is_invariant, deviation.get())
    
    return results

In [5]:
def test_rotation_invariance_parallel_gpu(images, max_degree, fractional_parameter_t, angles=[45, 90, 135, 180, 225, 270, 315, 360]):
    """
    Tests rotation invariance for multiple images in parallel using GPU and calculates deviations.
    
    Parameters:
        images (list of numpy arrays): List of images to test.
        max_degree (int): Maximum degree for the FrCHFM calculation.
        fractional_parameter_t (float): Fractional parameter for FrCHFM.
        angles (list): List of angles to rotate each image for testing.
        
    Returns:
        overall_results (dict): A dictionary where keys are image indices and values are dictionaries
                                of angle results, each with (is_invariant, deviation).
    """
    overall_results = {}
    with ThreadPoolExecutor() as executor:
        futures = {
            executor.submit(rotate_and_compare_gpu, img, max_degree, fractional_parameter_t, angles): idx
            for idx, img in enumerate(images)
        }
        
        # Use tqdm to show progress
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing Images"):
            idx = futures[future]
            try:
                result = future.result()
                overall_results[idx] = result
            except Exception as e:
                print(f"Image {idx} failed with exception: {e}")
    
    return overall_results


In [6]:
def test_convert_to_polar_rotation_consistency():
    image = cv2.imread('cs4485-images/prompt_0_image_0.png', cv2.IMREAD_GRAYSCALE)
    image_rotated_90 = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    
    # Convert both images to polar coordinates
    polar_image, r, theta = convert_to_polar(image)
    polar_image_rotated, r_rotated, theta_rotated = convert_to_polar(image_rotated_90)
    
    # Check if radius (r) and theta values are consistent
    assert np.allclose(r, r_rotated), "Radius values differ across rotations."
    assert np.allclose(theta, theta_rotated), "Theta values differ across rotations."
    print("test_convert_to_polar_rotation_consistency passed.")
    
test_convert_to_polar_rotation_consistency()

test_convert_to_polar_rotation_consistency passed.


In [7]:
def test_radial_basis_function_symmetry_invariance():
    image = cv2.imread('cs4485-images/prompt_0_image_0.png', cv2.IMREAD_GRAYSCALE)

    # Convert the original image to polar coordinates and extract radial values
    polar_image, r, theta = convert_to_polar(image)
    r_gpu = cp.array(r)

    # Compute the radial basis function for the original orientation
    Rtn_original = radial_basis_function_gpu(3, 0.5, r_gpu)

    # Define rotation angles for eight-way symmetry (in degrees)
    rotation_angles = [45, 90, 135, 180, 225, 270, 315, 360]

    for angle in rotation_angles:
        # Rotate the image and convert to polar coordinates
        rotated_image = cv2.rotate(image, angle)
        polar_image_rotated, r_rotated, _ = convert_to_polar(rotated_image)

        # Compute the radial basis function for the rotated orientation
        Rtn_rotated = radial_basis_function_gpu(3, 0.5, cp.array(r_rotated))

        # Check for consistent output between original and rotated radial basis functions
        assert np.allclose(Rtn_original.get(), Rtn_rotated.get(), atol=1e-3), \
            f"Radial basis function is not invariant for {angle}° rotation."

    print("test_radial_basis_function_symmetry_invariance passed.")

test_radial_basis_function_symmetry_invariance()

test_radial_basis_function_symmetry_invariance passed.


In [8]:
# Directory containing the images
image_folder = "cs4485-images"
# Parameters for FrCHFM calculation
max_degree = 5  # Change depending on desired precision
fractional_parameter_t = 0.5  # Fractional parameter t


# Load all images from the folder
images = [cv2.imread(os.path.join(image_folder, filename), cv2.IMREAD_GRAYSCALE)
          for filename in os.listdir(image_folder) if filename.endswith(('.png', '.jpg', '.jpeg'))][:6]

# Run the parallel rotation invariance test
#results = test_rotation_invariance_parallel_gpu(images, max_degree, fractional_parameter_t)
for i in range(1, 7):
    print("Max Degree: ",i,test_rotation_invariance_parallel_gpu(images, i, fractional_parameter_t))

# Print the results
#print(results)

Processing Images: 100%|█████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 18.23it/s]


Max Degree:  1 {1: {45: (False, array(1464960.14440767)), 90: (False, array(2705528.59009041)), 135: (False, array(1445102.50472404)), 180: (False, array(849254.1911503)), 225: (False, array(1478270.70952193)), 270: (False, array(2932154.19705877)), 315: (False, array(1427132.24407754)), 360: (True, array(0.))}, 3: {45: (False, array(1773087.20420404)), 90: (False, array(4438792.39071422)), 135: (False, array(1767608.83576397)), 180: (False, array(1124526.57564563)), 225: (False, array(1777285.26802967)), 270: (False, array(3961447.97734632)), 315: (False, array(1764657.87189682)), 360: (True, array(0.))}, 4: {45: (False, array(1913398.28827563)), 90: (False, array(4645379.6587614)), 135: (False, array(1893650.4471091)), 180: (False, array(1286158.55825844)), 225: (False, array(1924562.26569)), 270: (False, array(4427296.29527345)), 315: (False, array(1884937.01020976)), 360: (True, array(0.))}, 5: {45: (False, array(2632491.72954712)), 90: (False, array(3334252.90393558)), 135: (False

Processing Images: 100%|█████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 17.65it/s]


Max Degree:  2 {0: {45: (False, array(2468173.17188355)), 90: (False, array(2837250.87935813)), 135: (False, array(2491769.54339529)), 180: (False, array(2221990.9519612)), 225: (False, array(2471380.17042355)), 270: (False, array(4081400.87379171)), 315: (False, array(2511761.68406213)), 360: (True, array(0.))}, 3: {45: (False, array(2446305.32199574)), 90: (False, array(5916180.57895756)), 135: (False, array(2460354.1182006)), 180: (False, array(1707144.15572476)), 225: (False, array(2447329.48369341)), 270: (False, array(5920841.98649042)), 315: (False, array(2461123.0699429)), 360: (True, array(0.))}, 2: {45: (False, array(1990372.89983994)), 90: (False, array(7005800.41022846)), 135: (False, array(2006525.32946473)), 180: (False, array(1913364.85113807)), 225: (False, array(1988310.69783028)), 270: (False, array(7868637.72284445)), 315: (False, array(2023163.45631107)), 360: (True, array(0.))}, 4: {45: (False, array(1406481.55428919)), 90: (False, array(6592917.6045664)), 135: (Fa

Processing Images: 100%|█████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 12.49it/s]


...

Max Degree:  4 {3: {45: (False, array(2525725.77990267)), 90: (False, array(4177392.01518105)), 135: (False, array(2544364.78799752)), 180: (False, array(10452864.88585532)), 225: (False, array(2498070.87428215)), 270: (False, array(4237313.2859824)), 315: (False, array(2531632.836416)), 360: (True, array(0.))}, 5: {45: (False, array(3058793.8512468)), 90: (False, array(3151184.05315877)), 135: (False, array(3297109.65492904)), 180: (False, array(8078484.56675665)), 225: (False, array(3043204.66390406)), 270: (False, array(3119932.25942915)), 315: (False, array(3282870.006079)), 360: (True, array(0.))}, 0: {45: (False, array(2533548.44568785)), 90: (False, array(2664297.12746378)), 135: (False, array(2570230.53468483)), 180: (False, array(4305657.57226721)), 225: (False, array(2550902.57062302)), 270: (False, array(3827344.30261015)), 315: (False, array(2627475.48572596)), 360: (True, array(0.))}, 1: {45: (False, array(2027649.05948032)), 90: (False, array(3621820.147993)), 135: (False

Processing Images: 100%|█████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00,  7.94it/s]


Max Degree:  5 {4: {45: (False, array(1752855.64348764)), 90: (False, array(3159438.72687631)), 135: (False, array(1750439.9228952)), 180: (False, array(12108334.2867292)), 225: (False, array(1799444.28493205)), 270: (False, array(3762303.40134613)), 315: (False, array(1777926.44921371)), 360: (True, array(0.))}, 0: {45: (False, array(2471303.65096653)), 90: (False, array(2616284.24405334)), 135: (False, array(2591214.04047411)), 180: (False, array(5154355.40099041)), 225: (False, array(2508250.33478279)), 270: (False, array(4415646.12497546)), 315: (False, array(2578703.06284948)), 360: (True, array(0.))}, 1: {45: (False, array(1945186.99760449)), 90: (False, array(3057659.9628074)), 135: (False, array(2128936.0295643)), 180: (False, array(10161919.68954696)), 225: (False, array(2017323.62572922)), 270: (False, array(3287695.10821026)), 315: (False, array(2100981.78125286)), 360: (True, array(0.))}, 2: {45: (False, array(2101723.30803168)), 90: (False, array(3859438.99431021)), 135: (

Processing Images: 100%|█████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00,  6.32it/s]

Max Degree:  6 {0: {45: (False, array(2688586.66967583)), 90: (False, array(2922548.68979545)), 135: (False, array(3043623.63388633)), 180: (False, array(6379675.02545401)), 225: (False, array(2719137.07506575)), 270: (False, array(5378682.47854553)), 315: (False, array(3088357.40332647)), 360: (True, array(0.))}, 1: {45: (False, array(2294352.20161125)), 90: (False, array(3311434.97088771)), 135: (False, array(2552807.94003482)), 180: (False, array(12980345.31019087)), 225: (False, array(2300803.85532789)), 270: (False, array(3386718.92544003)), 315: (False, array(2459255.12535604)), 360: (True, array(0.))}, 5: {45: (False, array(4336615.86053813)), 90: (False, array(3200532.35163351)), 135: (False, array(4819650.90056309)), 180: (False, array(11635654.18820832)), 225: (False, array(4342731.74013197)), 270: (False, array(3307553.64558163)), 315: (False, array(4820970.26748175)), 360: (True, array(0.))}, 4: {45: (False, array(1926204.0742261)), 90: (False, array(3104474.46036288)), 135